# AutoKeras (0.4)
This notebook contains the experiments with AutoKeras, analyzing the quality and performance of the generated models and comparing them to handcrafted ones.

Main questions:
*   What does the structure of the generated networks look like?
*   How is the quality compared to handmade nets?

"Auto-Keras is an open source software library for automated machine learning (AutoML). It is developed by DATA Lab at Texas A&M University and community contributors. The ultimate goal of AutoML is to provide easily accessible deep learning tools to domain experts with limited data science or machine learning background. Auto-Keras provides functions to automatically search for architecture and hyperparameters of deep learning models." - *autokeras.com*

## Packages and Imports
The following section contains the packages that need to be installed and imported.

In google colab you have to **restart the environment** after installing autokeras.

In [0]:
!pip install autokeras==0.4.0 # Version 0.4

In [0]:
!python -V # Should output 3.6.* to work

In [0]:
import autokeras as ak
from autokeras.image.image_supervised import ImageClassifier
from autokeras.utils import pickle_from_file
import keras
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from google.colab import files

## Load data set
In the following sections you can load the data set you are going to use to test AutoKeras. The training data is contained in `x_train` and `y_train`, while the test data is in `x_test` and `y_test`.

### MNIST

In [0]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
data_set_name = "mnist"

### Breast Cancer

In [0]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
data_set_name = "breast_cancer"

### Iris

In [0]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
data_set_name = "iris"

## Create AutoKeras model

In [0]:
%tensorflow_version 1.x
# ImageClassifier
clf = ImageClassifier(verbose=True, augment=True)
clf.fit(x_train, y_train, time_limit=1 * 60 * 60) # 1 Hour
clf.final_fit(x_train, y_train, x_test, y_test, retrain=False, trainer_args={'max_no_improvement_num': 5})

### Export model

In [0]:
clf.export_autokeras_model(data_set_name + ".pkl")

In [0]:
# Download model
files.download(data_set_name + ".pkl")

### Load model

In [0]:
# Upload model
files.upload()

In [0]:
clf = pickle_from_file(data_set_name + ".pkl")

## Create model for comparison
**Work in Progress**

In [0]:
# Comparison model for MNIST
from keras import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.losses import mean_squared_error

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=x_train[0].shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer=Adam(), loss=mean_squared_error)

In [0]:
model_log = model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1)

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Visualize and Evaluate
The visualization is still an ugly workaround, but it works.

In [0]:
# Evaluate model
print(clf.evaluate(x_test, y_test))

In [0]:
torch_model = clf.cnn.best_model.produce_model() # For trained model
#torch_model = clf.graph.produce_model() # For portable models that got imported

In [0]:
# Print model structure
torch_model

In [0]:
# Save and download PyTorch model
torch.save(torch_model, data_set_name + ".pth")
files.download(data_set_name + ".pth")

Subsequently use [this website](https://lutzroeder.github.io/netron/) to visualize the model.

## References
https://autokeras.com/